## Initialize AQS Dataset

In [ ]:
pip install uszipcode

In [ ]:
#import libraries
import requests
import json
import time
import pandas as pd
from uszipcode import SearchEngine

Data from the Air Quality System (AQS) API is avaliavble for anyone to use as long as you sign up. Tables of variable description, variable values needed for service requests and required variables for specific requests.

https://aqs.epa.gov/aqsweb/documents/data_api.html

The API has the following limits imposed on request size:
- Length of time: end date must be in the same year as begin date (except monitors)
- Number of params: 5 max
- Limit the queries to 1mil rows of data
- Limit the frequency of queries; they request a 5 second pause between requests and to not make 10 requests per minute.

In [ ]:
#setting up credentials
email = 'jacyyang04@gmail.com'
key = 'ochreosprey79'
creds = 'email={}&key={}'.format(email, key)

Obtaining daily data summaries from CA, we need:
- email
- key
- base url (api endpoint)
- param
- bdate
- edate
- state

In [ ]:
#function sets up url and specific parameters needed to call api in year given

def api_url(year):
    #set up mmddyyyy format
    bdate = year + '0101'
    edate = year + '1231'
    date_range = 'bdate={}&edate={}'.format(bdate, edate)
    
    #PM2.5 is fine inhalable particles, with diameters that are generally 2.5 micrometers and smaller
    param = 'param=88101'
    
    #california code: 06
    state = 'state=06'
    
    base_url = 'https://aqs.epa.gov/data/api/dailyData/byState'
    
    #generates api request url
    url = '{}?{}&{}&{}&{}'.format(base_url, creds, param, date_range, state)
    
    return url

In [ ]:
#calls for api request and returns api in json
def request_aqs(year):
    #calls set_url function which returns api url
    response = requests.get(api_url(year))
    return response

In [ ]:
#set required years
years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021']

#will hold list of df
raw_frames = []

#calls api request functions for each year
for year in years:
    #calls request_aqs function and returns json
    response = request_aqs(year)
    
    raw_data = response.json()
    
    #grab values with Data key and turns dictionary into dataframe
    df = pd.DataFrame(raw_data['Data'])
    
    #add df to raw_frames list
    raw_frames.append(df)
    #AQS API size restrictions
    time.sleep(8)
    
print('Success!')

In [62]:
#create dataframe by concating all items in raw_frames list
raw_df = pd.concat(raw_frames)

In [146]:
aqs_df = raw_df.copy()

In [147]:
aqs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068291 entries, 0 to 18447
Data columns (total 31 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   state_code           1068291 non-null  object 
 1   county_code          1068291 non-null  object 
 2   site_number          1068291 non-null  object 
 3   parameter_code       1068291 non-null  object 
 4   poc                  1068291 non-null  int64  
 5   latitude             1068291 non-null  float64
 6   longitude            1068291 non-null  float64
 7   datum                1068291 non-null  object 
 8   parameter            1068291 non-null  object 
 9   sample_duration      1068291 non-null  object 
 10  pollutant_standard   891122 non-null   object 
 11  date_local           1068291 non-null  object 
 12  units_of_measure     1068291 non-null  object 
 13  event_type           1068291 non-null  object 
 14  observation_count    1068291 non-null  int64  
 15  

In [163]:
#reset index
aqs_df = aqs_df.reset_index()

In [155]:
columns_desired = ['date_local',
                   'county',
                   'city',
                   'latitude',
                   'longitude',
                   'aqi',
                   'units_of_measure',
                   'first_max_value',
                   'first_max_hour',
]

In [156]:
pd.options.display.max_columns = 35
aqs_df.head()

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration,pollutant_standard,date_local,units_of_measure,event_type,observation_count,observation_percent,validity_indicator,arithmetic_mean,first_max_value,first_max_hour,aqi,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,06,027,0002,88101,1,37.360684,-118.330783,WGS84,PM2.5 - Local Conditions,1 HOUR,None,2015-12-31,Micrograms/cubic meter (LC),None,24,100.0,Y,1.850000,5.6,17,NaN,181,Thermo Scientific TEOM 1400 FDMS or 1405 8500C...,White Mountain Research Center - Owens Valley Lab,"WMRC/NCORE, 3000 E. LINE ST., BISHOP, CA. 93514",California,Inyo,Not in a city,13860,"Bishop, CA",2016-03-02
1,06,027,0002,88101,1,37.360684,-118.330783,WGS84,PM2.5 - Local Conditions,1 HOUR,None,2015-12-30,Micrograms/cubic meter (LC),None,24,100.0,Y,2.500000,6.2,13,NaN,181,Thermo Scientific TEOM 1400 FDMS or 1405 8500C...,White Mountain Research Center - Owens Valley Lab,"WMRC/NCORE, 3000 E. LINE ST., BISHOP, CA. 93514",California,Inyo,Not in a city,13860,"Bishop, CA",2016-03-02
2,06,027,0002,88101,1,37.360684,-118.330783,WGS84,PM2.5 - Local Conditions,1 HOUR,None,2015-12-29,Micrograms/cubic meter (LC),None,24,100.0,Y,1.354167,4.9,23,NaN,181,Thermo Scientific TEOM 1400 FDMS or 1405 8500C...,White Mountain Research Center - Owens Valley Lab,"WMRC/NCORE, 3000 E. LINE ST., BISHOP, CA. 93514",California,Inyo,Not in a city,13860,"Bishop, CA",2016-03-02
3,06,027,0002,88101,1,37.360684,-118.330783,WGS84,PM2.5 - Local Conditions,1 HOUR,None,2015-12-28,Micrograms/cubic meter (LC),None,24,100.0,Y,1.658333,8.9,6,NaN,181,Thermo Scientific TEOM 1400 FDMS or 1405 8500C...,White Mountain Research Center - Owens Valley Lab,"WMRC/NCORE, 3000 E. LINE ST., BISHOP, CA. 93514",California,Inyo,Not in a city,13860,"Bishop, CA",2016-03-02
4,06,027,0002,88101,1,37.360684,-118.330783,WGS84,PM2.5 - Local Conditions,1 HOUR,None,2015-12-27,Micrograms/cubic meter (LC),None,24,100.0,Y,2.658333,13.5,22,NaN,181,Thermo Scientific TEOM 1400 FDMS or 1405 8500C...,White Mountain Research Center - Owens Valley Lab,"WMRC/NCORE, 3000 E. LINE ST., BISHOP, CA. 93514",California,Inyo,Not in a city,13860,"Bishop, CA",2016-03-02


In [164]:
#dropduplicates
aqs_df = aqs_df.drop_duplicates()

In [165]:
len(aqs_df)

1068291

In [166]:
#change date_local values to datetime object
aqs_df['date_local'] = pd.to_datetime(aqs_df['date_local'])

In [167]:
#create new dataframe wth desired columns
aqs_filter_df = aqs_df[columns_desired]

In [168]:
len(aqs_filter_df)

1068291

In [169]:
aqs_filter_df

,date_local,county,city,latitude,longitude,aqi,units_of_measure,first_max_value,first_max_hour
0,2015-12-31,Inyo,Not in a city,37.360684,-118.330783,NaN,Micrograms/cubic meter (LC),5.6,17
1,2015-12-30,Inyo,Not in a city,37.360684,-118.330783,NaN,Micrograms/cubic meter (LC),6.2,13
2,2015-12-29,Inyo,Not in a city,37.360684,-118.330783,NaN,Micrograms/cubic meter (LC),4.9,23
3,2015-12-28,Inyo,Not in a city,37.360684,-118.330783,NaN,Micrograms/cubic meter (LC),8.9,6
4,2015-12-27,Inyo,Not in a city,37.360684,-118.330783,NaN,Micrograms/cubic meter (LC),13.5,22
...,...,...,...,...,...,...,...,...,...
1068286,2021-01-02,Riverside,Banning,33.944713,-116.830074,7.0,Micrograms/cubic meter (LC),1.7,0
1068287,2021-01-01,Riverside,Banning,33.944713,-116.830074,5.0,Micrograms/cubic meter (LC),1.2,0
1068288,2021-01-01,Riverside,Banning,33.944713,-116.830074,5.0,Micrograms/cubic meter (LC),1.2,0
1068289,2021-01-01,Riverside,Banning,33.944713,-116.830074,5.0,Micrograms/cubic meter (LC),1.2,0


In [173]:
#save into json
aqs_filter_df.to_json('aqs_PM2.5_data_clean.json')

In [ ]:
#save into csv
aqs_filter_df.to_csv('aqs_PM2.5_data_clean.csv')

## Obtaining Zipcodes ##

**If we want zipcodes..**

In [171]:
#set library to access the bigger zipcode database
search = SearchEngine(simple_zipcode=False)

In [172]:
#attempt 1 in search for zipcode
attempt_1 = search.by

from uszipcode import Zipcode

# Search zipcode within 30 miles, ordered from closest to farthest
result = search.by_coordinates(39.122229, -77.133578, radius=30, returns=5)

AttributeError: 'SearchEngine' object has no attribute 'by'